---

_You are currently looking at **version 1.5** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

# Assignment 3 - More Pandas
This assignment requires more individual learning then the last one did - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

### Question 1 (20%)
Load the energy data from the file `Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']`

Convert `Energy Supply` to gigajoules (there are 1,000,000 gigajoules in a petajoule). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, 

e.g. 

`'Bolivia (Plurinational State of)'` should be `'Bolivia'`, 

`'Switzerland17'` should be `'Switzerland'`.

<br>

Next, load the GDP data from the file `world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

<br>

Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

*This function should return a DataFrame with 20 columns and 15 entries.*

In [6]:
import pandas as pd
import numpy as np

#### 1. Load data, get rid of unnecessaty info, change the column labels

In [7]:
energy=pd.read_excel('Energy Indicators.xls',header=None)
energy=energy.iloc[16:,2:]
energy=energy.loc[:244]
energy=energy.drop(17,axis=0)
header=energy.loc[16]
header[2]='Country'
header[5]='% Renewable'
energy=energy.iloc[1:,]
energy=energy.rename(columns=header)

#### 2. Convert Energy Supply to gigajoules and '...' to np.nan,

In [8]:
energy['Energy Supply']=energy['Energy Supply'].map(lambda x:x=='...' and np.nan or x*1000)
energy['Energy Supply per capita']=energy['Energy Supply per capita'].map(lambda x:x=='...' and np.nan or x)
energy['% Renewable']=energy['% Renewable'].map(lambda x:x=='...' and np.nan or x)

#### 3. Remove numbers and parenthesis in country names

In [9]:
# country names with parenthesis
p_country=[x for x in energy['Country'] if '(' in x ]
p_country

['Bolivia (Plurinational State of)',
 'Falkland Islands (Malvinas)',
 'Iran (Islamic Republic of)',
 'Micronesia (Federated States of)',
 'Sint Maarten (Dutch part)',
 'Venezuela (Bolivarian Republic of)']

In [10]:
# removing parenthesis
for i,country in enumerate(energy['Country']):
    if '(' in country:
        country=country.split('(')[0].strip()
        energy['Country'].iloc[i]=country

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [11]:
# country names with numbers
num_list=['1','2','3','4','5','6','7','8','9','0']
num_country=[x for x in energy['Country'] if x[-1] in num_list ]
num_country

['Australia1',
 'China2',
 'China, Hong Kong Special Administrative Region3',
 'China, Macao Special Administrative Region4',
 'Denmark5',
 'France6',
 'Greenland7',
 'Indonesia8',
 'Italy9',
 'Japan10',
 'Kuwait11',
 'Netherlands12',
 'Portugal13',
 'Saudi Arabia14',
 'Serbia15',
 'Spain16',
 'Switzerland17',
 'Ukraine18',
 'United Kingdom of Great Britain and Northern Ireland19',
 'United States of America20']

In [12]:
# removing numbers
for i,country in enumerate(energy['Country']):
    if country[-1] in num_list:
        country=''.join([x for x in country if not x.isdigit()])
        energy['Country'].iloc[i]=country

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


#### 4.Rename country names

In [13]:
def country_name_energy(name):
    if name=='Republic of Korea':
        name="South Korea"
    elif name=="United Kingdom of Great Britain and Northern Ireland":
        name="United Kingdom"
    elif name=='United States of America':
        name="United States"
    elif name=="China, Hong Kong Special Administrative Region":
        name='Hong Kong'
    return name

energy['Country']=energy['Country'].map(lambda x:country_name_energy(x))


##### 5. Load Gdp data, change columns names and country names

In [14]:
GDP = pd.read_csv('world_bank.csv',header=None,skiprows=4)
header_GDP=GDP.iloc[0]
header_GDP=header_GDP.map(lambda x:int(x) if type(x)==np.float64 else x)
GDP=GDP.iloc[1:,]
GDP=GDP.rename(columns=header_GDP)

def country_name_gdp(name):
    if name=="Korea, Rep.":
        name="South Korea"
    elif name=="Iran, Islamic Rep.":
        name="Iran"
    elif name=="Hong Kong SAR, China":
        name="Hong Kong"
    return name

GDP['Country Name']=GDP['Country Name'].map(lambda x:country_name_gdp(x))
GDP=GDP.rename(columns={'Country Name':'Country'})

In [15]:
GDP.head()

,Country,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
1,Aruba,ABW,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.467704e+09,NaN,NaN,NaN,NaN,NaN
2,Andorra,AND,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,4.018196e+09,4.021331e+09,3.675728e+09,3.535389e+09,3.346317e+09,3.185605e+09,3.129538e+09,3.127550e+09,NaN,NaN
3,Afghanistan,AFG,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,1.030523e+10,1.172119e+10,1.214448e+10,1.469733e+10,1.593680e+10,1.691113e+10,1.935220e+10,1.973134e+10,1.999032e+10,2.029415e+10
4,Angola,AGO,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,5.581103e+10,6.842044e+10,7.787420e+10,7.975320e+10,8.247091e+10,8.570262e+10,9.012096e+10,9.626143e+10,1.008863e+11,1.039106e+11
5,Albania,ALB,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,9.771760e+09,1.034829e+10,1.112752e+10,1.150029e+10,1.192695e+10,1.223109e+10,1.240477e+10,1.254247e+10,1.279331e+10,1.312082e+10


#### 6.  Load journal data and merge 3 datasets

In [16]:
ScimEn=pd.read_excel('scimagojr-3.xlsx')
ScimEn=ScimEn.iloc[:15,:]

year_drop=[i for i in range(1960,2006)]
columns_to_drop=year_drop+['Country Code','Indicator Name','Indicator Code']
GDP2=GDP.drop(columns_to_drop,axis=1)


merged1=pd.merge(energy,GDP2)
merged_df=pd.merge(ScimEn,merged1)
merged_df=merged_df.set_index('Country')
merged_df.head()

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per capita,% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,,,,,,,,
China,1,127050,126767,597237,411683,4.70,138,127191000.0,93.0,19.75491,3.992331e+12,4.559041e+12,4.997775e+12,5.459247e+12,6.039659e+12,6.612490e+12,7.124978e+12,7.672448e+12,8.230121e+12,8.797999e+12
United States,2,96661,94747,792274,265436,8.20,230,90838000.0,286.0,11.57098,1.479230e+13,1.505540e+13,1.501149e+13,1.459484e+13,1.496437e+13,1.520402e+13,1.554216e+13,1.577367e+13,1.615662e+13,1.654857e+13
Japan,3,30504,30287,223024,61554,7.31,134,18984000.0,149.0,10.23282,5.496542e+12,5.617036e+12,5.558527e+12,5.251308e+12,5.498718e+12,5.473738e+12,5.569102e+12,5.644659e+12,5.642884e+12,5.669563e+12
United Kingdom,4,20944,20357,206091,37874,9.84,139,7920000.0,124.0,10.60047,2.419631e+12,2.482203e+12,2.470614e+12,2.367048e+12,2.403504e+12,2.450911e+12,2.479809e+12,2.533370e+12,2.605643e+12,2.666333e+12
Russian Federation,5,18534,18301,34266,12422,1.85,57,30709000.0,214.0,17.28868,1.385793e+12,1.504071e+12,1.583004e+12,1.459199e+12,1.524917e+12,1.589943e+12,1.645876e+12,1.666934e+12,1.678709e+12,1.616149e+12


In [17]:
def answer_one():
    return merged_df

### Question 2 (6.6%)
The previous question joined three datasets then reduced this to just the top 15 entries. When you joined the datasets, but before you reduced this to the top 15 items, how many entries did you lose?

*This function should return a single number.*

In [18]:
%%HTML
<svg width="800" height="300">
  <circle cx="150" cy="180" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="blue" />
  <circle cx="200" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="red" />
  <circle cx="100" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="green" />
  <line x1="150" y1="125" x2="300" y2="150" stroke="black" stroke-width="2" fill="black" stroke-dasharray="5,3"/>
  <text  x="300" y="165" font-family="Verdana" font-size="35">Everything but this!</text>
</svg>

In [19]:
def answer_two():
    ans=(energy.shape[0]-merged1.shape[0])+(GDP.shape[0]-merged1.shape[0])
    +(merged1.shape[0]-merged_df.shape[0])

    return ans
answer_two()

119

## Answer the following questions in the context of only the top 15 countries by Scimagojr Rank (aka the DataFrame returned by `answer_one()`)

### Question 3 (6.6%)
What is the average GDP over the last 10 years for each country? (exclude missing values from this calculation.)

*This function should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [115]:
def answer_three():
    gdp_10years=merged_df.iloc[:,10:]
    avgGDP=gdp_10years.mean(axis=1).sort_values(ascending=False)
    return avgGDP

### Question 4 (6.6%)
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*

In [116]:
def answer_four():
    avgGDP[avgGDP==avgGDP.iloc[5]].index.tolist()
    ans=gdp_10years.ix[3,2015]-gdp_10years.ix[3,2006]
    return ans

### Question 5 (6.6%)
What is the mean `Energy Supply per Capita`?

*This function should return a single number.*

In [57]:
def answer_five():
    Top15 = answer_one()
    ans=Top15['Energy Supply per capita'].mean()
    return ans
answer_five()

157.6

### Question 6 (6.6%)
What country has the maximum % Renewable and what is the percentage?

*This function should return a tuple with the name of the country and the percentage.*

In [74]:
def answer_six():
    Top15 = answer_one()
    country=Top15['% Renewable'].sort_values(ascending=False).index[0]
    percentage=Top15['% Renewable'].sort_values(ascending=False).iloc[0]
    ans=(country,percentage)
    return ans
answer_six()

('Brazil', 69.64803)

### Question 7 (6.6%)
Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This function should return a tuple with the name of the country and the ratio.*

In [27]:
def answer_seven():
    Top15 = answer_one()
    Top15['Citation_ratio']=Top15['Self-citations']/Top15['Citations']
    ratio=Top15['Citation_ratio'].sort_values(ascending=False)
    ans=(ratio.index[0],ratio[0])
    return ans
answer_seven()

('China', 0.6893126179389422)

### Question 8 (6.6%)

Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
What is the third most populous country according to this estimate?

*This function should return a single string value.*

In [31]:
def answer_eight():
    Top15 = answer_one()
    Top15['Population']=Top15['Energy Supply']/Top15['Energy Supply per capita']
    ans=Top15['Population'].sort_values(ascending=False).index[2]
    return ans
answer_eight()

'United States'

### Question 9 (6.6%)
Create a column that estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the `.corr()` method, (Pearson's correlation).

*This function should return a single number.*

*(Optional: Use the built-in function `plot9()` to visualize the relationship between Energy Supply per Capita vs. Citable docs per Capita)*

In [38]:
def answer_nine():
    Top15 = answer_one()
    Top15['Citable docs per capita']=Top15['Citable documents']/Top15['Population']
    ans=Top15['Citable docs per capita'].corr(Top15['Energy Supply per capita'])
    return ans
answer_nine()

0.7940010435442941

In [46]:
def plot9():
    import matplotlib as plt
    %matplotlib inline
    
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per capita']
    Top15['Citable docs per Capita'] = Top15['Citable documents'] / Top15['PopEst']
    Top15.plot(x='Citable docs per Capita', y='Energy Supply per capita', kind='scatter', xlim=[0, 0.0006])

In [48]:
# plot9() # Be sure to comment out plot9() before submitting the assignment!

### Question 10 (6.6%)
Create a new column with a 1 if the country's % Renewable value is at or above the median for all countries in the top 15, and a 0 if the country's % Renewable value is below the median.

*This function should return a series named `HighRenew` whose index is the country name sorted in ascending order of rank.*

In [53]:
def answer_ten():
    Top15 = answer_one()
    Top15['HighRenew']=Top15['% Renewable'].map(lambda x: 1 if x>= Top15['% Renewable'].median() else 0)
    
    return Top15['HighRenew']
answer_ten()

Country
China                 1
United States         0
Japan                 0
United Kingdom        0
Russian Federation    1
Canada                1
Germany               1
India                 0
France                1
South Korea           0
Italy                 1
Spain                 1
Iran                  0
Australia             0
Brazil                1
Name: HighRenew, dtype: int64

### Question 11 (6.6%)
Use the following dictionary to group the Countries by Continent, then create a dateframe that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

In [99]:
 ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}

def answer_eleven():
    Top15 = answer_one()   
    Top15=Top15.reset_index()
    Top15['Continent']=Top15['Country'].map(lambda x:ContinentDict[x] )
    ans=Top15.groupby('Continent')['Population'].aggregate({'size':'count','sum':'sum','mean':'mean','std':'std'})

    return ans

### Question 12 (6.6%)
Cut % Renewable into 5 bins. Group Top15 by the Continent, as well as these new % Renewable bins. How many countries are in each of these groups?

*This function should return a __Series__ with a MultiIndex of `Continent`, then the bins for `% Renewable`. Do not include groups with no countries.*

In [102]:
def answer_twelve():
    Top15 = answer_one()
    Top15=Top15.reset_index()
    Top15['Continent']=Top15['Country'].map(lambda x:ContinentDict[x] )
    Top15['Renewable_binned']=pd.cut(Top15['% Renewable'],5)
    ans=Top15.groupby(['Continent','Renewable_binned'])['Country'].count()
    return ans

### Question 13 (6.6%)
Convert the Population Estimate series to a string with thousands separator (using commas). Do not round the results.

e.g. 317615384.61538464 -> 317,615,384.61538464

*This function should return a Series `PopEst` whose index is the country name and whose values are the population estimate string.*

In [ ]:
def answer_thirteen():    
    Top15=answer_one()
    Top15['Population']=Top15['Energy Supply']/Top15['Energy Supply per capita']
    int_list=[int(i) for i in Top15['Population']]
    ans=[format(i,',') for i in int_list]
    return ans

### Optional

Use the built in function `plot_optional()` to see an example visualization.

In [112]:
def plot_optional():
    import matplotlib as plt
    %matplotlib inline
    Top15 = answer_one()
    ax = Top15.plot(x='Rank', y='% Renewable', kind='scatter', 
                    c=['#e41a1c','#377eb8','#e41a1c','#4daf4a','#4daf4a','#377eb8','#4daf4a','#e41a1c',
                       '#4daf4a','#e41a1c','#4daf4a','#4daf4a','#e41a1c','#dede00','#ff7f00'], 
                    xticks=range(1,16), s=6*Top15['2014']/10**10, alpha=.75, figsize=[16,6]);

    for i, txt in enumerate(Top15.index):
        ax.annotate(txt, [Top15['Rank'][i], Top15['% Renewable'][i]], ha='center')

    print("This is an example of a visualization that can be created to help understand the data. \
This is a bubble chart showing % Renewable vs. Rank. The size of the bubble corresponds to the countries' \
2014 GDP, and the color corresponds to the continent.")

In [114]:
#plot_optional() # Be sure to comment out plot_optional() before submitting the assignment!